In [15]:
pip install openai


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Only if needed for rust
! sudo xcodebuild -license accept

In [14]:
# If not installed
! brew install rust

==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Auto-updated Homebrew!
Updated 5 taps (ngrok/ngrok, datadog/tap, withgraphite/tap, homebrew/core and homebrew/cask).
==> New Formulae
atomic_queue: C++14 lock-free queues
attempt-cli: CLI for retrying fallible commands
bedtk: Simple toolset for BED files
claude-cmd: Claude Code Commands Manager
claude-hooks: Hook system for Claude Code
codebook-lsp: Code-aware spell checker language server
context7-mcp: Up-to-date code documentation for LLMs and AI code editors
dash-mpd-cli: Download media content from a DASH-MPEG or DASH-WebM MPD manifest
docker-compose-langserver: Language service for Docker Compose documents
dqlite: Embeddable, replicated and fault-tolerant SQLite-powered engine
fjira: Fuzzy-find cli jira interface
gitingest: Turn any Git repository into a prompt-friendly text

In [15]:
# Build the dd-trace-py from source. It takes a few minutes to build/install
pip install git+https://github.com/datadog/dd-trace-py@2927ae70451d41206e8d6abb2c3b24d3816a954d

  Cloning https://github.com/datadog/dd-trace-py (to revision 2927ae70451d41206e8d6abb2c3b24d3816a954d) to /private/var/folders/fw/0099t8y529b2x_y472l_vnr40000gp/T/pip-req-build-nbsene08
  Running command git clone --filter=blob:none --quiet https://github.com/datadog/dd-trace-py /private/var/folders/fw/0099t8y529b2x_y472l_vnr40000gp/T/pip-req-build-nbsene08
  Running command git rev-parse -q --verify 'sha^2927ae70451d41206e8d6abb2c3b24d3816a954d'
  Running command git fetch -q https://github.com/datadog/dd-trace-py 2927ae70451d41206e8d6abb2c3b24d3816a954d
  Running command git checkout -q 2927ae70451d41206e8d6abb2c3b24d3816a954d
  Resolved https://github.com/datadog/dd-trace-py to commit 2927ae70451d41206e8d6abb2c3b24d3816a954d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ddtrace: filename=ddtrace-3.15.0.dev62+g2927ae704-cp311-cp311-macosx_15_0_arm64.whl size=8247167 sha256=95

In [37]:
from ddtrace.llmobs import LLMObs
from ddtrace.llmobs.utils import Prompt
from ddtrace import patch
from openai import OpenAI
import openai
from dotenv import load_dotenv
import os

load_dotenv()
patch(openai=True)

openai.api_key = os.environ.get("openai_api_key")
client = OpenAI()
load_dotenv()

LLMObs.enable(
    api_key=os.environ.get("dd_api_key"),
    site=os.environ.get("dd_site"),
    ml_app="prompt_tracking_bug_bash",
    service="shopist-simple",
    agentless_enabled=True,
    env="testing",
    integrations_enabled=False,
)

recommendation_prompt_id = "recommendation-prompt"

In [38]:
def query_openai(system_prompt=None, user_prompt=None, variables=[], model="gpt-4o", max_tokens=100, temperature=0.7):
     """ Sends a prompt to the OpenAI API and returns the text response. """ 
     if system_prompt and user_prompt:
          system_prompt_rendered = system_prompt.format(**variables)
          user_prompt_rendered = user_prompt.format(**variables)
          completion = client.chat.completions.create(
               model=model,
               max_tokens=max_tokens,
               temperature=temperature,
               messages=[{
                    "role": "system",
                    "content": system_prompt_rendered
                    },
                    {
                    "role": "user",
                    "content": user_prompt_rendered
                    }]
          )
     elif user_prompt:
          user_prompt_rendered = user_prompt.format(**variables)
          completion = client.chat.completions.create(
               model=model,
               max_tokens=max_tokens,
               temperature=temperature,
               messages=[{
                    "role": "user",
                    "content": user_prompt_rendered
                    }]
          )
     elif system_prompt:
            system_prompt_rendered = system_prompt.format(**variables)
            completion = client.chat.completions.create(
                 model=model,
                 max_tokens=max_tokens,
                 temperature=temperature,
                 messages=[{
                        "role": "system",
                        "content": system_prompt_rendered
                        }]
            )
     return completion.choices[0].message.content

# How to annotate a prompt to a span 
In order to trace the prompt we wrap the openai call with LLMObs.annotation_context.
Example :

```
with LLMObs.annotation_context(prompt=Prompt(id=prompt_id,
                           chat_template=[("User", prompt_step1)],
                           variables=variables_step1
                           )):
```

# Step 1 simple user prompt
Prompt for a recommendation system

In [47]:
prompt_step1 = "{user_prompt}"
variables_step1 = {"user_prompt": "Suggest a destination and a restaurant."}

for i in range(6):
    with LLMObs.annotation_context(prompt=Prompt(id=recommendation_prompt_id,
                           template=prompt_step1,
                           variables=variables_step1
                           )):
        response_step1 = query_openai(user_prompt=prompt_step1, variables=variables_step1)

print("=== USER Prompt Template (Step 1) ===") 
print(prompt_step1) 
print("\n=== Variables (Step 1) ===") 
print(variables_step1)
print("\n=== Response (Step 1) ===") 
print(response_step1)

=== USER Prompt Template (Step 1) ===
{user_prompt}

=== Variables (Step 1) ===
{'user_prompt': 'Suggest a destination and a restaurant.'}

=== Response (Step 1) ===
If you're looking for a memorable travel experience, consider visiting Kyoto, Japan. This city is rich in history and culture, offering beautiful temples, traditional tea houses, and stunning gardens.

While in Kyoto, dine at **Kikunoi**, a renowned kaiseki restaurant that offers a multi-course Japanese meal, exquisitely prepared with seasonal ingredients. The experience at Kikunoi is not just about the food, but also about the presentation and the ambiance, reflecting the elegance of Japanese hospitality. Be sure


# Step 2 add examples
Second version of the prompt that implements few shot prompt for the recommendation system

In [48]:
prompt_step2 = """
    {user_prompt}
    Examples:
        {examples}
"""

# Examples
examples_step2 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step2 = {
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step2
    }

for i in range(2):
    with LLMObs.annotation_context(prompt=Prompt(id=recommendation_prompt_id,
                           template=prompt_step2,
                           variables=variables_step2
                           )):
        response_step2 = query_openai(user_prompt=prompt_step2, variables=variables_step2)

print("=== USER Prompt template (Step 2) ===") 
print(prompt_step2) 
print("\n=== Variables (Step 2) ===")
print(variables_step2)
print("\n=== Response (Step 2) ===") 
print(response_step2)

=== USER Prompt template (Step 2) ===

    {user_prompt}
    Examples:
        {examples}


=== Variables (Step 2) ===
{'user_prompt': 'Suggest a destination and a restaurant.', 'examples': '\nExample 1:\nInput: "Bob: Can you suggest a restaurant?"\nOutput: "Based on your preferences for Italian cuisine, I recommend trying Luigi\'s Trattoria."\n\nExample 2:\nInput: "Alice: What\'s a good place for nature walks?"\nOutput: "Considering your love for scenic views, I suggest visiting Green Park Gardens."\n'}

=== Response (Step 2) ===
Input: "John: I'm looking for a vibrant city experience with a great place to eat."  
Output: "For a lively urban adventure, I recommend visiting New York City. Be sure to dine at Katz's Delicatessen for an authentic and iconic meal experience."


# Step 3 add system prompt with constraints and examples and lighten the user prompt

In [49]:
system_prompt_step3 = """
            You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
            Here are some rules and examples:

            - Always base your responses on user preferences.
            - Do not provide answers beyond the scope of the context given.

            Example exchanges:
            {examples}

            Use these examples to format your responses. 

            {constraints}

            Now, here's your specific task:
    """

prompt_step3 = "{user_prompt}"

examples_step3 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step3 = {
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step3, 
    "constraints": "Do not ask the user for more information. Either provide a recommendation or state that data is insufficient for a recommendation."
    }

for i in range(3):
    with LLMObs.annotation_context(prompt=Prompt(id=recommendation_prompt_id,
                           chat_template=[{"role":"System","content":system_prompt_step3},{"role":"User", "content":prompt_step3}],
                           variables=variables_step3
                           )):
        response_step3 = query_openai(system_prompt=system_prompt_step3, user_prompt=prompt_step3, variables=variables_step3)

print("=== SYSTEM Prompt template (Step 3) ===")
print(system_prompt_step3)
print("=== USER Prompt template (Step 3) ===") 
print(prompt_step3)
print("\n=== Variables (Step 3) ===")
print(variables_step3)
print("\n=== Response (Step 3) ===") 
print(response_step3)

=== SYSTEM Prompt template (Step 3) ===

            You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
            Here are some rules and examples:

            - Always base your responses on user preferences.
            - Do not provide answers beyond the scope of the context given.

            Example exchanges:
            {examples}

            Use these examples to format your responses. 

            {constraints}

            Now, here's your specific task:
    
=== USER Prompt template (Step 3) ===
{user_prompt}

=== Variables (Step 3) ===
{'user_prompt': 'Suggest a destination and a restaurant.', 'examples': '\nExample 1:\nInput: "Bob: Can you suggest a restaurant?"\nOutput: "Based on your preferences for Italian cuisine, I recommend trying Luigi\'s Trattoria."\n\nExample 2:\nInput: "Alice: What\'s a good place for nature walks?"\nOutput: "Considering your love for sceni

# Step 4 add context to system prompt and name of the user

In [50]:
system_prompt_step4 = """
        You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
        Here are some rules and examples:

        - Always base your responses on user preferences.
        - Do not provide answers beyond the scope of the context given.

        Example exchanges:
        {examples}

        Knowing the users' preferences is key to providing accurate recommendations. Here is the context for the current user:
        {name}'s Context: {context}

        Use these examples to format your responses. 

        {constraints}

        Now, here's your specific task:
    """

prompt_step4 = "{user_prompt}"

examples_step4 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step4 = {
    "name": "Alice",
    "context": "Alice is allergic to peanuts and enjoys traveling to beaches and mountains. She prefers vegetarian food.",
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step4, 
    "constraints": "Do not ask the user for more information. Either provide a recommendation or state that data is insufficient for a recommendation."
    }

for i in range(4):
    with LLMObs.annotation_context(prompt=Prompt(id=recommendation_prompt_id,
                           chat_template=[{"role":"System","content":system_prompt_step4},{"role":"User", "content":prompt_step4}],
                           variables=variables_step4
                           )):
        response_step4 = query_openai(system_prompt=system_prompt_step4, user_prompt=prompt_step4, variables=variables_step4)

print("=== SYSTEM Prompt template (Step 4) ===")
print(system_prompt_step4)
print("=== USER Prompt template (Step 4) ===") 
print(prompt_step4)
print("\n=== Variables (Step 4) ===")
print(variables_step4)
print("\n=== Response (Step 4) ===") 
print(response_step4)

=== SYSTEM Prompt template (Step 4) ===

        You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
        Here are some rules and examples:

        - Always base your responses on user preferences.
        - Do not provide answers beyond the scope of the context given.

        Example exchanges:
        {examples}

        Knowing the users' preferences is key to providing accurate recommendations. Here is the context for the current user:
        {name}'s Context: {context}

        Use these examples to format your responses. 

        {constraints}

        Now, here's your specific task:
    
=== USER Prompt template (Step 4) ===
{user_prompt}

=== Variables (Step 4) ===
{'name': 'Alice', 'context': 'Alice is allergic to peanuts and enjoys traveling to beaches and mountains. She prefers vegetarian food.', 'user_prompt': 'Suggest a destination and a restaurant.', 'examples': '\n

In [ ]:
system_prompt_step5 = """
    You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences while maintaining the given context. Here are the rules:
    - Base recommendations primarily on user preferences.
    - Respect and adhere strictly to the context boundaries; do not go beyond the provided scope.
    - Personalization is key to every response.

    Knowing the users' preferences is key to providing accurate recommendations. Here is the context for the current user:
        {name}'s Context: {context}

    Example exchanges: {examples}

    Apply these examples to guide your response format, ensuring a focus on personal preferences within context. {constraints} 
    Now, here's your specific task:
    """

prompt_step5 = "{user_prompt}"

examples_step5 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step5 = {
    "name": "Alice",
    "context": "Alice is allergic to peanuts and enjoys traveling to beaches and mountains. She prefers vegetarian food.",
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step5, 
    "constraints": "Do not ask the user for more information. Either provide a recommendation or state that data is insufficient for a recommendation."
    }

for i in range(5):
    with LLMObs.annotation_context(prompt=Prompt(id=recommendation_prompt_id,
                           chat_template=[{"role":"System","content":system_prompt_step5},{"role":"User", "content":prompt_step5}],
                           variables=variables_step5
                           )):
        response_step5 = query_openai(system_prompt=system_prompt_step5, user_prompt=prompt_step5, variables=variables_step5)

print("=== SYSTEM Prompt template (Step 5) ===")
print(system_prompt_step5)
print("=== USER Prompt template (Step 5) ===") 
print(prompt_step5)
print("\n=== Variables (Step 5) ===")
print(variables_step5)
print("\n=== Response (Step 5) ===") 
print(response_step5)

## ShopistApp : System prompt + User Prompt

In [46]:
system_prompt = """
You are a support agent at a furniture store meant to help users with their requests.
Shopist has the following policies:
    {policies}
"""

user_prompt = "{user_prompt}"

policies =  """Shopist Policies:

Returns & Exchanges:
Items can be returned or exchanged within 30 days with receipt, if unused. Custom orders cannot be returned.

Warranty:
Most items have a 1-year warranty for manufacturer defects. Damage from use is not covered.

Delivery & Assembly:
Delivery is available for a fee. Report any damage within 48 hours of delivery.

Pricing & Promotions:
Sale prices and discounts can’t be combined. We match sale prices within 7 days of your purchase.

Cancellations:
Orders may be cancelled free within 24 hours. After that, fees may apply.

Support Hours:
We’re here to help Monday–Saturday, 9am–6pm."""

variables = {"policies":policies}
for i in range(3):
    with LLMObs.annotation_context(prompt=Prompt(id="shopist_app",
                           template=system_prompt,
                           variables=variables
                           )):
        response = query_openai(system_prompt=system_prompt, user_prompt=user_prompt, variables={"policies":policies, "user_prompt":"What is the return policy?"})

print("=== USER Prompt Template (Step 1) ===")
print(system_prompt, user_prompt)
print("\n=== Variables (Step 1) ===")
print(variables)
print("\n=== Response (Step 1) ===")
print(response)

=== USER Prompt Template (Step 1) ===

You are a support agent at a furniture store meant to help users with their requests.
Shopist has the following policies:
    {policies}
 {user_prompt}

=== Variables (Step 1) ===
{'policies': 'Shopist Policies:\n\nReturns & Exchanges:\nItems can be returned or exchanged within 30 days with receipt, if unused. Custom orders cannot be returned.\n\nWarranty:\nMost items have a 1-year warranty for manufacturer defects. Damage from use is not covered.\n\nDelivery & Assembly:\nDelivery is available for a fee. Report any damage within 48 hours of delivery.\n\nPricing & Promotions:\nSale prices and discounts can’t be combined. We match sale prices within 7 days of your purchase.\n\nCancellations:\nOrders may be cancelled free within 24 hours. After that, fees may apply.\n\nSupport Hours:\nWe’re here to help Monday–Saturday, 9am–6pm.'}

=== Response (Step 1) ===
Items can be returned or exchanged within 30 days with a receipt, as long as they are unused. 

## ShopistApp : Improve the system prompt

In [45]:
system_prompt = """
You are a helpful support agent at a furniture store meant to help users with their requests.

Shopist has the following policies:
{policies}

Be kind, solution-oriented, and follow company policy—but always try to leave the customer feeling heard and satisfied.
Respond to the user's request by providing information about the policies above.
"""

user_prompt = "{user_prompt}"

policies =  """Shopist Policies:

Returns & Exchanges:
Items can be returned or exchanged within 30 days with receipt, if unused. Custom orders cannot be returned.

Warranty:
Most items have a 1-year warranty for manufacturer defects. Damage from use is not covered.

Delivery & Assembly:
Delivery is available for a fee. Report any damage within 48 hours of delivery.

Pricing & Promotions:
Sale prices and discounts can’t be combined. We match sale prices within 7 days of your purchase.

Cancellations:
Orders may be cancelled free within 24 hours. After that, fees may apply.

Support Hours:
We’re here to help Monday–Saturday, 9am–6pm."""

variables = {"policies":policies}
for i in range(5):
    with LLMObs.annotation_context(prompt=Prompt(id="shopist_app",
                           template=system_prompt,
                           variables=variables
                           )):
        response = query_openai(system_prompt=system_prompt, user_prompt=user_prompt, variables={"policies":policies, "user_prompt":"What is the return policy?"})

print("=== USER Prompt Template (Step 1) ===")
print(system_prompt, user_prompt)
print("\n=== Variables (Step 1) ===")
print(variables)
print("\n=== Response (Step 1) ===")
print(response)

=== USER Prompt Template (Step 1) ===

You are a helpful support agent at a furniture store meant to help users with their requests.

Shopist has the following policies:
{policies}

Be kind, solution-oriented, and follow company policy—but always try to leave the customer feeling heard and satisfied.
Respond to the user's request by providing information about the policies above.
 {user_prompt}

=== Variables (Step 1) ===
{'policies': 'Shopist Policies:\n\nReturns & Exchanges:\nItems can be returned or exchanged within 30 days with receipt, if unused. Custom orders cannot be returned.\n\nWarranty:\nMost items have a 1-year warranty for manufacturer defects. Damage from use is not covered.\n\nDelivery & Assembly:\nDelivery is available for a fee. Report any damage within 48 hours of delivery.\n\nPricing & Promotions:\nSale prices and discounts can’t be combined. We match sale prices within 7 days of your purchase.\n\nCancellations:\nOrders may be cancelled free within 24 hours. After th